# Протоколы обмена ключами

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

In [3]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

In [4]:
def randint(a, b):
    """
    Генерация слуайного числа среди заданных.
    """
    
    native_a = int(a)
    native_b = int(b)
    return gmpy2.mpz(random.randint(native_a, native_b))

----

In [5]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime
from Crypto.Random import random

# HKDF и то, что надо для нее
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA512

# Подпись и шифрование
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Cipher import AES

# Математическая библиотека
import gmpy2

# Для генерации идентификаторов
import uuid

# Утилитки
from Crypto.Util.number import long_to_bytes
import binascii

----

## Протокол Диффи-Хеллмана
![Diffe-Hellman Protocol](./images/Diffie-Hellman_4.png)

In [6]:
class DiffieHellmanParams(object):
    def __init__(self, prime_bits: int = 512):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[DiffieHellmanParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [7]:
class DiffieHellmanUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: DiffieHellmanParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        self._sabotage = use_another_value
        
        trace('[DiffieHellmanUser]', f'client {self._id} created')
    
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, отправляю другому пользователю
        # Получаю от него g ** y mod p
        #
        
        x = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{x = }')
        
        intermediate_value = another_client._accept_key_exchange(gmpy2.powmod(self._g, x, self._p))
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[DiffieHellmanUser]', f'[{self._id}]', f'guessed {x = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value) -> int:
        #
        # Получаю g ** x mod p, генерирую у
        #
        
        y = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{y = }')
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return gmpy2.powmod(self._g, y, self._p)
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [8]:
# Параметры протокола
params = DiffieHellmanParams()

[TRACE] [DiffieHellmanParams] p = 13050454453319207048771856410408478277532176597076838058571976309591082690961044326177020187140770384184987556561894973067272726341385572722824073305245481, g = 2846560203099132610509227562546992691291050617577646705435316780859888719401126279318914678524682903733349240929043482706673695338670534767501481539478631


In [9]:
# Два хороших пользователя
alice = DiffieHellmanUser(params)
bob   = DiffieHellmanUser(params)

[TRACE] [DiffieHellmanUser] client 1eec4553-694a-4604-bf83-85d847cd6beb created
[TRACE] [DiffieHellmanUser] client 5efbeb7e-95db-4686-b116-83c1b9fac7d5 created


In [10]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [1eec4553-694a-4604-bf83-85d847cd6beb] x = mpz(7590380024611895823296116345800574276338880234583508452293952336663972027559632414342548778802235474780390560116058781010156558649773262801299035506540322)
[TRACE] [DiffieHellmanUser] [5efbeb7e-95db-4686-b116-83c1b9fac7d5] y = mpz(12735767799682693808885816551733386932312244862154996455665742380244171932112039540266994967352121018488482824545699139656366217725813271030604524955092723)
[TRACE] [DiffieHellmanUser] [5efbeb7e-95db-4686-b116-83c1b9fac7d5] derived key: e439a50fd16f7b2f4589e4b88e79885c8ccf84b79cc510db8f86d24954376dd3
[TRACE] [DiffieHellmanUser] [1eec4553-694a-4604-bf83-85d847cd6beb] derived key: e439a50fd16f7b2f4589e4b88e79885c8ccf84b79cc510db8f86d24954376dd3


In [11]:
# Теперь врага создам
mallory = DiffieHellmanUser(params, True)

[TRACE] [DiffieHellmanUser] client a02f8286-44d4-4c99-89e6-8b28c967e18a created


In [12]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [a02f8286-44d4-4c99-89e6-8b28c967e18a] x = mpz(142334018006864260618591972107885279532717711745059795439729018249993142103951338561739772281831447764890047580735844383522102226371942177170530268605820)
[TRACE] [DiffieHellmanUser] [5efbeb7e-95db-4686-b116-83c1b9fac7d5] y = mpz(901748609758730405211553494083587267422314046051667606380774360166575518499300221280884472279841282566845226751417969421278756042041451581363259457943961)
[TRACE] [DiffieHellmanUser] [5efbeb7e-95db-4686-b116-83c1b9fac7d5] derived key: 8d1bddd6bdf751af103422b566b0ccc72c150b8ebff1eae97eefb72508a63092
[TRACE] [DiffieHellmanUser] [a02f8286-44d4-4c99-89e6-8b28c967e18a] guessed x = mpz(6265816291259633955208821050969311008681473878456145139232779838941270294208851148760583210634095513460592517180794065930839489958808862988293704968419437)
[TRACE] [DiffieHellmanUser] [a02f8286-44d4-4c99-89e6-8b28c967e18a] derived key: 725bf9fd2d7b0b08d7354184461c970a7dc4098d3be98e7bbce6091e2821a604


----
## Протокол MTI
![Diffe-Hellman Protocol](./images/MTI_4.png)

In [13]:
class MTIParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[MTIParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [14]:
class MTIUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: MTIParams, *,
                 fake_data: gmpy2.mpz = None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = self._generate_group_element()
        self._public_key  = gmpy2.powmod(self._g, self._private_key, self._p) if fake_data is None else fake_data
        
        trace('[MTIUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
        
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому клиенту и получаю от него g ** y mod p
        #
        
        intermediate_value = another_client._accept_key_exchange(m, self.public_key)
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(another_client.public_key, x, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
    
    
    def _accept_key_exchange(self, intermediate_value, public_key):
        #
        # Генерирую у, вычисляю g ** y mod p
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        #
        # Вывожу ключ
        #
        
        product = gmpy2.powmod(public_key, y, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(MTIUser.HKDF_DUMMY_STUFF, 
                         MTIUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return m
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [15]:
# Параметры протокола
params = MTIParams(512)

[TRACE] [MTIParams] p = 10239616951841482776890332367544202909245870286638284242892950742964402988663552859095603262762622655322336632008050756463625732553916159787690930592472007, g = 3865777710395288142916275541718365774554825144705928483270209675517557407818264292862652889642384532790261896750832037701943450659893339051534676923325881


In [16]:
# Два хороших пользователя
alice = MTIUser(params)
bob   = MTIUser(params)

[TRACE] [MTIUser] client 3039d041-d24b-4c0a-867f-af12e72b6fe4 created. PK = 2876649882695933487127112523733324370070318611287180921185632756570891561537257603983264099834647206320618752172043677493465618732868080681670756708305171, SK = 4213191982845276559314311611603912668831659920160129432959299509061005629268304179704140318620008482827417731907633491505450575530461607678140346330710319
[TRACE] [MTIUser] client 847ebe0a-7303-49df-aee2-d567fda2a994 created. PK = 2355666381794413074251432517803979918406604837578287644029505597792108538262473259231595958312083829785565959666399894918657009780768640642355748520043812, SK = 6050550252136353556256420933661428953010258833603541569116570148956642841727931741893801385959345840435481967758552275775189874032104535997039764775009703


In [17]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [MTIUser] [3039d041-d24b-4c0a-867f-af12e72b6fe4] x = mpz(7070152387452466714454483392633748746840602123049813524679606356582052474572351997860778311143502456890525552072676435151390914000494777168022831819061955), m = mpz(838568069491091766733944913020291390121433094790412274800338769968860648496689593353223661631795577947146068303137375227822155595050899276081903044291455)
[TRACE] [MTIUser] [847ebe0a-7303-49df-aee2-d567fda2a994] y = mpz(350685276830303220518871705578447595795273809420651729858714590226996375786920240949605575125810424546889209908588330891083173452904882673642374254158258), m = mpz(7249977022612992695232120711620601967194948366907770109904484587322434011646295517724181569742141142189555206508544965482363895364491512663494264019310422)
[TRACE] [MTIUser] [847ebe0a-7303-49df-aee2-d567fda2a994] derived key: d834d4ee43bb04718da12d1ecd6035f12800abb748accf942eee88af2aebc24c
[TRACE] [MTIUser] [3039d041-d24b-4c0a-867f-af12e72b6fe4] derived key: d834d4ee43bb04718da12d1ec

In [18]:
# Теперь врага создам
mallory = MTIUser(params, fake_data=alice.public_key)

[TRACE] [MTIUser] client 231f789e-bcc0-4f1a-be45-ee9de4d46bc3 created. PK = 2876649882695933487127112523733324370070318611287180921185632756570891561537257603983264099834647206320618752172043677493465618732868080681670756708305171, SK = 1814727410981337866865193958621376644564972255966319578778748045409886528466065032160798530550661487690406963273947005657381073602053057100690181109297752


In [19]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [MTIUser] [231f789e-bcc0-4f1a-be45-ee9de4d46bc3] x = mpz(6196341805758526088407953959332818733069478697105112077007376389054077430183018877882188263056044313779048821932585456387906576841363928833904285185152853), m = mpz(4457413157876661843207839231620174757172024507373650965801064076568708696405024090084991233791195330764152685807302378543358336287557580157819025440686613)
[TRACE] [MTIUser] [847ebe0a-7303-49df-aee2-d567fda2a994] y = mpz(9125254082374244926417879424600695438813653056142771315317961257848340156220603703809839929266058940088231553706662438915542679530123461119279797772055131), m = mpz(8230168569209810094554527055078961656551779454811094546908498875418205998206771145337986266844875142107050523135435889779260136029595700589718876066938251)
[TRACE] [MTIUser] [847ebe0a-7303-49df-aee2-d567fda2a994] derived key: f795bc59309122eaff24db5f4410a902ae1e25bfb66d1ba65047db7a965a03c5
[TRACE] [MTIUser] [231f789e-bcc0-4f1a-be45-ee9de4d46bc3] derived key: 116609842eef7dc6a8d3d16

----
## Протокол STS
![STS Protocol](./images/STS_4.png)

In [20]:
class STSParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[STSParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [21]:
class STSUser(object):
    RSA_KEY_LENGTH = 4096
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: STSParams, *, 
                 fake_data=None):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = RSA.generate(STSUser.RSA_KEY_LENGTH)
        self._public_key  = self._private_key.publickey() if fake_data is None else fake_data
        
        trace('[STSUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        #
        # Генерирую х, вычисляю g ** x mod p
        #
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        #
        # Отправляю другому пользователю, получаю g ** y mod p и зашифрованную подпись
        #
        
        intermediate_value, encrypted_signature = another_client._accept_key_exchange(m)
        
        #
        # Вывожу ключ
        #
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Расшифровываю подпись и проверяю ее
        # Если не подошла, то ключ неверный, завершаюсь
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(intermediate_value, m, received_signature, another_client.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return
        
        #
        # Сам теперь подписываю значения и отправляю на проверку другому пользователю
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        if not self._check_key_exchange(m, intermediate_value, 
                                        (cipher.nonce, cipher.encrypt(signature)), self.public_key):
            trace('[STSUser]', f'[{self._id}]', 'Another user refused key exchange')
            self._key = None
            return
        
        #
        # Тут все хорошо
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value):
        #
        # Генерирую у, вычисляю g ** y mod p
        # Вывожу ключ
        #
        
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[STSUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        #
        # Создаю подпись значений и отправляю инициатору
        #
        
        signature = self._sign_values(m, intermediate_value)
        
        cipher = AES.new(self._key, AES.MODE_CTR)
        encrypted_signature = (cipher.nonce, cipher.encrypt(signature))
        
        return m, encrypted_signature
    
    
    def _check_key_exchange(self, f, s, encrypted_signature, pk):
        #
        # Проверяю подпись от инициатора
        #
        
        nonce, ct = encrypted_signature
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        received_signature = cipher.decrypt(ct)
        
        if not self._check_signature(f, s, received_signature, pk):
            trace('[STSUser]', f'[{self._id}]', 'Invalid signature')
            self._key = None
            return False
        
        #
        # Если все валидно, то ключ верно установлен
        #
        
        trace('[STSUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        return True
        
    
    def _sign_values(self, f, s):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        return pss.new(self._private_key).sign(h)
    
    
    def _check_signature(self, f, s, signature, pk):
        first  = long_to_bytes(int(f))
        second = long_to_bytes(int(s))
        
        h = SHA512.new(first + second)
        verifier = pss.new(pk)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
    
        
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)
        
    
    @property
    def public_key(self):
        """
        Получение открытого ключа.
        """
        
        return self._public_key

-----

In [22]:
# Параметры протокола
params = STSParams(512)

[TRACE] [STSParams] p = 10475680594941275897006354674200670422557574240865885285525312259520156971317989715798391283012665689363651101824918850929904176174379213792523797247620923, g = 1448603152343160426311195695699719764685088013643771755724126833807221857893548501185899849799179971328575652611713978142416186592376564175508708285328257


In [23]:
# Два хороших пользователя
alice = STSUser(params)
bob   = STSUser(params)

[TRACE] [STSUser] client fe3c1106-7552-4d22-b1b1-7084415350aa created. PK = Public RSA key at 0x1063C83A0, SK = Private RSA key at 0x1065F3B80
[TRACE] [STSUser] client 97b98e26-b809-4e19-b826-e10077c098d8 created. PK = Public RSA key at 0x10675D190, SK = Private RSA key at 0x1065F3730


In [24]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [STSUser] [fe3c1106-7552-4d22-b1b1-7084415350aa] x = mpz(9175749596004743785615012783639464631987260508428303665107146390270324682126645406939810897967247548800325044082592250708288856197429074348058043944420147), m = mpz(8814676452648698085058725170816234596030196142090341455532236541132984537350253190930996018963040157077149459650413316172047956007459905445756962153348091)
[TRACE] [STSUser] [97b98e26-b809-4e19-b826-e10077c098d8] y = mpz(1336181523612163798928515747191801746231828428905319357965602352862767358085122973095044791216275472941019065848106537944873005220294100067548959010448222), m = mpz(4736553953965072395130437858578813510285421187730615391919909404050393341237542767272918420509565716738573342116834691429993026797288841184044666862147364)
[TRACE] [STSUser] [fe3c1106-7552-4d22-b1b1-7084415350aa] derived key: c33797ff22694f6fe1d024ef9231cb1a2ec388e70ff52cc516da5d74bcf99e03
[TRACE] [STSUser] [fe3c1106-7552-4d22-b1b1-7084415350aa] derived key: c33797ff22694f6fe1d024e

In [25]:
# Теперь врага создам
mallory = STSUser(params, fake_data=alice.public_key)

[TRACE] [STSUser] client c1bb9193-25d4-4018-9169-d1845cb47a9c created. PK = Public RSA key at 0x1063C83A0, SK = Private RSA key at 0x1065F4B50


In [26]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [STSUser] [c1bb9193-25d4-4018-9169-d1845cb47a9c] x = mpz(5984173386049415695536015139384749483219976293594350447084614041184459449800981439625146824112721797275742050863385233893336432070484475673891555314088131), m = mpz(4326718731315345542984123694747621034417123239321594398725448069522168486094191698940148354368327385977992017068123341885746122067282797357094487621667578)
[TRACE] [STSUser] [97b98e26-b809-4e19-b826-e10077c098d8] y = mpz(6004943798912262290163657064449885138326767116132696977530574770784458356735820822045567102249892290771090509097130433220151425054906217461310870227833034), m = mpz(5688613908384658927538179059683740236440293170431480047050811069428643370917902754725385363558700489447928137064540500963860333226886763212596527298673509)
[TRACE] [STSUser] [c1bb9193-25d4-4018-9169-d1845cb47a9c] Invalid signature
[TRACE] [STSUser] [c1bb9193-25d4-4018-9169-d1845cb47a9c] Another user refused key exchange


In [27]:
bob.initiate_key_exchange(mallory)

[TRACE] [STSUser] [97b98e26-b809-4e19-b826-e10077c098d8] x = mpz(2232618758525617128983625279351860010927931009800584918009105283595363635372307272270679891862406828795735846567124064953236162556996128481879602714459430), m = mpz(1660676111819737750545139900735373695607110696674428168089566850117045012416057600520481175817156781159201423027625533866730971411240490785403799896228860)
[TRACE] [STSUser] [c1bb9193-25d4-4018-9169-d1845cb47a9c] y = mpz(3308732720681852331377391958908351460682776336815548858040101999804130505456047278193983603438030234299569916405295712852497735709794699607327188809017284), m = mpz(7266318727392245297468133508285448525871413050081517522259560696777677466573046388190355376710198923359922440531773677234564306156929428010161898312145239)
[TRACE] [STSUser] [97b98e26-b809-4e19-b826-e10077c098d8] Invalid signature
